# Data Cleaning Pipeline - Phase 2: The Rosetta Stone

**Goal:** Merge MatchHistory, Understat, and ClubElo into a single Master Training Set.

| Step | Description |
|------|-------------|
| Cell 1 | Setup, load raw data, define team name mappings |
| Cell 2 | Apply mappings and validate team alignment |
| Cell 3 | Merge MatchHistory + Understat (on date + home/away teams) |
| Cell 4 | Merge ClubElo (nearest-date Elo lookup for home and away) |
| Cell 5 | Validation and quality checks |
| Cell 6 | Save Master_Training_Set.csv |

In [11]:
# Ensure working directory is the project root perfectly across IDEs/Terminals
import os
import sys
try:
    if 'notebooks' in os.getcwd():
        project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
    else:
        project_root = os.getcwd()
    os.chdir(project_root)
    if project_root not in sys.path:
        sys.path.append(project_root)
except Exception:
    pass

# =============================================================================
# Cell 1: Setup, Load Raw Data, Define Team Name Mappings
# =============================================================================

import pandas as pd
import numpy as np
import os
import glob
import warnings
warnings.filterwarnings('ignore')

raw_dir = os.path.join('data', 'raw')
processed_dir = os.path.join('data', 'processed')
os.makedirs(processed_dir, exist_ok=True)

# --- Load all MatchHistory files into one DataFrame ---
print('Loading MatchHistory data...')
mh_files = sorted(glob.glob(os.path.join(raw_dir, '*_MatchHistory.csv')))
mh_frames = []
for f in mh_files:
    df = pd.read_csv(f)
    mh_frames.append(df)
    print(f'  {os.path.basename(f):45s} {len(df):>5} rows')
mh_all = pd.concat(mh_frames, ignore_index=True)
print(f'  Total MatchHistory: {len(mh_all):,} rows')

# --- Load all Understat files into one DataFrame ---
print('\nLoading Understat data...')
us_files = sorted(glob.glob(os.path.join(raw_dir, '*_Understat.csv')))
us_frames = []
for f in us_files:
    df = pd.read_csv(f)
    us_frames.append(df)
    print(f'  {os.path.basename(f):45s} {len(df):>5} rows')
us_all = pd.concat(us_frames, ignore_index=True)
print(f'  Total Understat: {len(us_all):,} rows')

# --- Load ClubElo ---
print('\nLoading ClubElo data...')
elo_all = pd.read_csv(os.path.join(raw_dir, 'ClubElo_Master.csv'))
print(f'  ClubElo: {len(elo_all):,} rows, {elo_all["team"].nunique()} unique teams')

# =========================================================================
# TEAM NAME MAPPINGS
# All names are standardized TO MatchHistory format (our source of truth).
# =========================================================================

# --- Understat -> MatchHistory ---
understat_to_mh = {
    # ENG - Premier League
    'Manchester City':           'Man City',
    'Manchester United':         'Man United',
    'Newcastle United':          'Newcastle',
    'Nottingham Forest':         "Nott'm Forest",
    'West Bromwich Albion':      'West Brom',
    'Wolverhampton Wanderers':   'Wolves',
    'Queens Park Rangers':       'QPR',
    # ESP - La Liga
    'Athletic Club':             'Ath Bilbao',
    'Atletico Madrid':           'Ath Madrid',
    'Real Betis':                'Betis',
    'Celta Vigo':                'Celta',
    'Espanyol':                  'Espanol',
    'SD Huesca':                 'Huesca',
    'Deportivo La Coruna':       'La Coruna',
    'Real Sociedad':             'Sociedad',
    'Sporting Gijon':            'Sp Gijon',
    'Real Valladolid':           'Valladolid',
    'Rayo Vallecano':            'Vallecano',
    # GER - Bundesliga
    'Arminia Bielefeld':         'Bielefeld',
    'Borussia Dortmund':         'Dortmund',
    'Eintracht Frankfurt':       'Ein Frankfurt',
    'FC Cologne':                'FC Koln',
    'Fortuna Duesseldorf':       'Fortuna Dusseldorf',
    'Hamburger SV':              'Hamburg',
    'Hannover 96':               'Hannover',
    'FC Heidenheim':             'Heidenheim',
    'Hertha Berlin':             'Hertha',
    'Bayer Leverkusen':          'Leverkusen',
    'Borussia M.Gladbach':       "M'gladbach",
    'Mainz 05':                  'Mainz',
    'Nuernberg':                 'Nurnberg',
    'RasenBallsport Leipzig':    'RB Leipzig',
    'St. Pauli':                 'St Pauli',
    'VfB Stuttgart':             'Stuttgart',
    # ITA - Serie A
    'AC Milan':                  'Milan',
    'SPAL 2013':                 'Spal',
    'Parma Calcio 1913':         'Parma',
    # FRA - Ligue 1
    'GFC Ajaccio':               'Ajaccio GFCO',
    'SC Bastia':                 'Bastia',
    'Clermont Foot':             'Clermont',
    'Paris Saint Germain':       'Paris SG',
    'Saint-Etienne':             'St Etienne',
}

# --- ClubElo -> MatchHistory ---
clubelo_to_mh = {
    'Atletico':          'Ath Madrid',
    'Bilbao':            'Ath Bilbao',
    'Bayern':            'Bayern Munich',
    'Frankfurt':         'Ein Frankfurt',
    'Espanyol':          'Espanol',
    'Gladbach':          "M'gladbach",
    'Forest':            "Nott'm Forest",
    'Koeln':             'FC Koln',
    'Duesseldorf':       'Fortuna Dusseldorf',
    'Holstein':          'Holstein Kiel',
    'Depor':             'La Coruna',
    'Nuernberg':         'Nurnberg',
    'Gijon':             'Sp Gijon',
    'Rayo Vallecano':    'Vallecano',
    'Werder':            'Werder Bremen',
    'Evian TG':          'Evian Thonon Gaillard',
    'Schalke':           'Schalke 04',
}

print(f'\nMappings loaded: {len(understat_to_mh)} Understat, {len(clubelo_to_mh)} ClubElo')
print('Setup complete.')

Loading MatchHistory data...
  ENG_Premier_League_MatchHistory.csv            4180 rows
  ESP_La_Liga_MatchHistory.csv                   4180 rows
  FRA_Ligue_1_MatchHistory.csv                   3931 rows
  GER_Bundesliga_MatchHistory.csv                3366 rows
  ITA_Serie_A_MatchHistory.csv                   4180 rows
  Total MatchHistory: 19,837 rows

Loading Understat data...
  ENG_Premier_League_Understat.csv               4180 rows
  ESP_La_Liga_Understat.csv                      4180 rows
  FRA_Ligue_1_Understat.csv                      4032 rows
  GER_Bundesliga_Understat.csv                   3366 rows
  ITA_Serie_A_Understat.csv                      4180 rows
  Total Understat: 19,938 rows

Loading ClubElo data...
  ClubElo: 699,162 rows, 161 unique teams

Mappings loaded: 42 Understat, 17 ClubElo
Setup complete.


In [12]:
# =============================================================================
# Cell 2: Apply Name Mappings & Validate Team Alignment
# =============================================================================

# --- Apply Understat mappings ---
print('Applying Understat name mappings...')
us_all['home_team'] = us_all['home_team'].replace(understat_to_mh)
us_all['away_team'] = us_all['away_team'].replace(understat_to_mh)

# --- Apply ClubElo mappings ---
print('Applying ClubElo name mappings...')
elo_all['team'] = elo_all['team'].replace(clubelo_to_mh)

# --- Validation: check orphans after mapping ---
mh_teams = set(mh_all['home_team'].dropna().unique()) | set(mh_all['away_team'].dropna().unique())
us_teams = set(us_all['home_team'].dropna().unique()) | set(us_all['away_team'].dropna().unique())
elo_teams = set(str(t) for t in elo_all['team'].dropna().unique())

# Filter out team codes (3-letter) and IDs (numeric) from Understat, but keep ones matching MH
us_team_names = {t for t in us_teams if (len(str(t)) > 3 and not str(t).isdigit()) or t in mh_teams}

mh_not_us = sorted(mh_teams - us_team_names)
mh_not_elo = sorted(mh_teams - elo_teams)

print(f'\n--- Post-Mapping Orphan Check ---')
print(f'  MH teams total: {len(mh_teams)}')
print(f'  Understat team names: {len(us_team_names)}')
print(f'  ClubElo teams: {len(elo_teams)}')

if mh_not_us:
    print(f'\n  [WARN] {len(mh_not_us)} MH teams still missing from Understat:')
    for t in mh_not_us:
        print(f'    - {t}')
else:
    print(f'\n  [OK] All MH teams found in Understat')

if mh_not_elo:
    print(f'\n  [WARN] {len(mh_not_elo)} MH teams still missing from ClubElo:')
    for t in mh_not_elo:
        print(f'    - {t}')
else:
    print(f'\n  [OK] All MH teams found in ClubElo')

print('\nName alignment complete.')

Applying Understat name mappings...
Applying ClubElo name mappings...

--- Post-Mapping Orphan Check ---
  MH teams total: 163
  Understat team names: 163
  ClubElo teams: 161

  [OK] All MH teams found in Understat

  [WARN] 2 MH teams still missing from ClubElo:
    - Ajaccio GFCO
    - St Etienne

Name alignment complete.


In [13]:
# =============================================================================
# Cell 3: Merge MatchHistory + Understat
# =============================================================================

print('Preparing data for merge...')

# --- Normalize dates to YYYY-MM-DD for both datasets ---
mh_all['date_norm'] = pd.to_datetime(mh_all['date'], format='mixed').dt.date
us_all['date_norm'] = pd.to_datetime(us_all['date'], format='mixed').dt.date

# --- Select key columns from Understat ---
us_cols = ['date_norm', 'home_team', 'away_team', 'home_xg', 'away_xg']
us_merge = us_all[us_cols].copy()

# --- Drop duplicate Understat rows (same date + teams) ---
us_merge = us_merge.drop_duplicates(subset=['date_norm', 'home_team', 'away_team'])
print(f'  Understat merge-ready: {len(us_merge):,} rows')

# --- Merge ---
print('Merging MatchHistory + Understat on [date, home_team, away_team]...')
merged = mh_all.merge(
    us_merge,
    on=['date_norm', 'home_team', 'away_team'],
    how='left',
    suffixes=('', '_us')
)

# --- Report merge quality ---
xg_matched = merged['home_xg'].notna().sum()
xg_pct = xg_matched / len(merged) * 100
print(f'\n  Total rows: {len(merged):,}')
print(f'  xG matched: {xg_matched:,} / {len(merged):,} ({xg_pct:.1f}%)')
print(f'  xG missing: {merged["home_xg"].isna().sum():,} ({100-xg_pct:.1f}%)')

# --- Per-league breakdown ---
print(f'\n  Per-league xG match rates:')
for league in sorted(merged['league'].unique()):
    mask = merged['league'] == league
    total = mask.sum()
    matched = merged.loc[mask, 'home_xg'].notna().sum()
    pct = matched / total * 100 if total > 0 else 0
    status = '[OK]  ' if pct >= 95 else '[WARN]'
    print(f'    {status} {league:25s} {matched:>5}/{total:>5} ({pct:.1f}%)')

print('\nMH + Understat merge complete.')

Preparing data for merge...
  Understat merge-ready: 18,112 rows
Merging MatchHistory + Understat on [date, home_team, away_team]...

  Total rows: 19,837
  xG matched: 19,694 / 19,837 (99.3%)
  xG missing: 143 (0.7%)

  Per-league xG match rates:
    [OK]   ENG-Premier League         4154/ 4180 (99.4%)
    [OK]   ESP-La Liga                4112/ 4180 (98.4%)
    [OK]   FRA-Ligue 1                3896/ 3931 (99.1%)
    [OK]   GER-Bundesliga             3365/ 3366 (100.0%)
    [OK]   ITA-Serie A                4167/ 4180 (99.7%)

MH + Understat merge complete.


In [14]:
# =============================================================================
# Cell 4: Merge ClubElo (Nearest-Date Elo Lookup)
# =============================================================================

print('Preparing ClubElo data for nearest-date merge...')

# --- Parse the ClubElo date column ---
# ClubElo has a 'to' column = end date of that Elo rating period
elo_all['elo_date'] = pd.to_datetime(elo_all['to'], format='mixed', errors='coerce')
elo_clean = elo_all.dropna(subset=['elo_date', 'team', 'elo']).copy()
elo_clean = elo_clean[['team', 'elo_date', 'elo']].copy()
elo_clean = elo_clean.sort_values('elo_date').reset_index(drop=True)
print(f'  ClubElo records after cleaning: {len(elo_clean):,}')

# --- Convert match dates for merge_asof ---
merged['match_date'] = pd.to_datetime(merged['date_norm'])

# --- Home Elo lookup ---
print('Looking up Home team Elo ratings...')
home_df = merged[['match_date', 'home_team']].copy()
home_df = home_df.rename(columns={'home_team': 'team'})
home_df = home_df.sort_values('match_date').reset_index()

home_elo = pd.merge_asof(
    home_df,
    elo_clean.rename(columns={'elo': 'home_elo'}),
    left_on='match_date',
    right_on='elo_date',
    by='team',
    direction='backward'
)
home_elo = home_elo.set_index('index')['home_elo']
merged['home_elo'] = home_elo

# --- Away Elo lookup ---
print('Looking up Away team Elo ratings...')
away_df = merged[['match_date', 'away_team']].copy()
away_df = away_df.rename(columns={'away_team': 'team'})
away_df = away_df.sort_values('match_date').reset_index()

away_elo = pd.merge_asof(
    away_df,
    elo_clean.rename(columns={'elo': 'away_elo'}),
    left_on='match_date',
    right_on='elo_date',
    by='team',
    direction='backward'
)
away_elo = away_elo.set_index('index')['away_elo']
merged['away_elo'] = away_elo

# --- Derived feature ---
merged['elo_diff'] = merged['home_elo'] - merged['away_elo']

# --- Report ---
elo_matched = merged['home_elo'].notna().sum()
elo_pct = elo_matched / len(merged) * 100
print(f'\n  Home Elo matched: {elo_matched:,} / {len(merged):,} ({elo_pct:.1f}%)')
print(f'  Away Elo matched: {merged["away_elo"].notna().sum():,} / {len(merged):,}')
print(f'  Elo missing: {merged["home_elo"].isna().sum():,} ({100-elo_pct:.1f}%)')

# --- Per-league breakdown ---
print(f'\n  Per-league Elo match rates:')
for league in sorted(merged['league'].unique()):
    mask = merged['league'] == league
    total = mask.sum()
    matched = merged.loc[mask, 'home_elo'].notna().sum()
    pct = matched / total * 100 if total > 0 else 0
    status = '[OK]  ' if pct >= 95 else '[WARN]'
    print(f'    {status} {league:25s} {matched:>5}/{total:>5} ({pct:.1f}%)')

# --- Drop helper columns ---
merged = merged.drop(columns=['match_date'], errors='ignore')

print('\nClubElo merge complete.')

Preparing ClubElo data for nearest-date merge...
  ClubElo records after cleaning: 699,162
Looking up Home team Elo ratings...
Looking up Away team Elo ratings...

  Home Elo matched: 19,654 / 19,837 (99.1%)
  Away Elo matched: 19,654 / 19,837
  Elo missing: 183 (0.9%)

  Per-league Elo match rates:
    [OK]   ENG-Premier League         4180/ 4180 (100.0%)
    [OK]   ESP-La Liga                4180/ 4180 (100.0%)
    [OK]   FRA-Ligue 1                3748/ 3931 (95.3%)
    [OK]   GER-Bundesliga             3366/ 3366 (100.0%)
    [OK]   ITA-Serie A                4180/ 4180 (100.0%)

ClubElo merge complete.


In [15]:
# =============================================================================
# Cell 5: Validation & Quality Checks
# =============================================================================

print('MASTER DATASET VALIDATION')
print('=' * 60)

# --- Key columns to validate ---
key_cols = ['home_xg', 'away_xg', 'home_elo', 'away_elo', 'elo_diff']
threshold = 0.05  # 5% max missing

print(f'\n  Missingness check (threshold: <{threshold*100:.0f}%):')
all_pass = True
for col in key_cols:
    if col in merged.columns:
        missing = merged[col].isna().sum()
        pct = missing / len(merged)
        passed = pct < threshold
        status = '[PASS]' if passed else '[FAIL]'
        if not passed:
            all_pass = False
        print(f'    {status} {col:15s} {missing:>6} missing ({pct*100:.2f}%)')
    else:
        print(f'    [FAIL] {col:15s} COLUMN NOT FOUND')
        all_pass = False

# --- League coverage ---
print(f'\n  League coverage:')
for league in sorted(merged['league'].unique()):
    mask = merged['league'] == league
    rows = mask.sum()
    seasons = merged.loc[mask, 'season'].nunique()
    print(f'    {league:25s} {rows:>5} rows, {seasons:>2} seasons')

# --- Shape summary ---
print(f'\n  Dataset shape: {merged.shape[0]:,} rows x {merged.shape[1]} columns')
print(f'  Date range: {merged["date_norm"].min()} to {merged["date_norm"].max()}')

# --- Sample output ---
print(f'\n  Sample columns in final dataset:')
sample_cols = ['league', 'season', 'date_norm', 'home_team', 'away_team',
               'FTHG', 'FTAG', 'FTR', 'home_xg', 'away_xg',
               'home_elo', 'away_elo', 'elo_diff', 'B365H', 'B365D', 'B365A']
available = [c for c in sample_cols if c in merged.columns]
print(merged[available].head(3).to_string())

# --- Final verdict ---
print(f'\n{"=" * 60}')
if all_pass:
    print('  VALIDATION PASSED -- All key columns below 5% missing data.')
else:
    print('  VALIDATION FAILED -- Some columns exceed 5% missing threshold.')
    print('  Review the mapping dictionaries and re-run.')

MASTER DATASET VALIDATION

  Missingness check (threshold: <5%):
    [PASS] home_xg            143 missing (0.72%)
    [PASS] away_xg            143 missing (0.72%)
    [PASS] home_elo           183 missing (0.92%)
    [PASS] away_elo           183 missing (0.92%)
    [PASS] elo_diff           364 missing (1.83%)

  League coverage:
    ENG-Premier League         4180 rows, 10 seasons
    ESP-La Liga                4180 rows, 10 seasons
    FRA-Ligue 1                3931 rows, 10 seasons
    GER-Bundesliga             3366 rows, 10 seasons
    ITA-Serie A                4180 rows, 10 seasons

  Dataset shape: 19,837 rows x 163 columns
  Date range: 2014-08-08 to 2025-05-25

  Sample columns in final dataset:
               league  season   date_norm   home_team       away_team  FTHG  FTAG FTR  home_xg   away_xg     home_elo     away_elo    elo_diff  B365H  B365D  B365A
0  ENG-Premier League    1415  2014-08-16     Arsenal  Crystal Palace   2.0   1.0   H  1.55411  0.158151  1865.238892

In [16]:
# =============================================================================
# Cell 6: Save Master Training Set
# =============================================================================

output_path = os.path.join(processed_dir, 'Master_Training_Set.csv')

# --- Drop the temporary date_norm if we want to keep original date ---
# Keep date_norm as it is cleaner for downstream use
merged.to_csv(output_path, index=False)

size_mb = os.path.getsize(output_path) / (1024 * 1024)
print(f'Master Training Set saved:')
print(f'  Path:    {os.path.abspath(output_path)}')
print(f'  Rows:    {len(merged):,}')
print(f'  Columns: {merged.shape[1]}')
print(f'  Size:    {size_mb:.1f} MB')
print(f'\nPhase 2 complete. Ready for model training.')

Master Training Set saved:
  Path:    c:\Users\ljega\Downloads\MSBA\Sports Analytics Project\data\processed\Master_Training_Set.csv
  Rows:    19,837
  Columns: 163
  Size:    11.4 MB

Phase 2 complete. Ready for model training.
